In [1]:
import os
import re
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.parse import quote
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from operator import eq

In [2]:
def getBs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs = BeautifulSoup(html, 'html.parser')
    except AttributeError as e:
        return None
    return bs

In [3]:
main_url = ('https://movie.naver.com/movie/sdb/browsing/')
bs = getBs('https://movie.naver.com/movie/sdb/browsing/bmovie_nation.nhn') #
# https://movie.naver.com/movie/sdb/browsing/bmovie_nation.nhn 제작 국가
# https://movie.naver.com/movie/sdb/browsing/bmovie_year.nhn 제작 년도
# https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn 개봉 년도
# https://movie.naver.com/movie/sdb/browsing/bmovie_genre.nhn 장르
# https://movie.naver.com/movie/sdb/browsing/bmovie_form.nhn 형식
# https://movie.naver.com/movie/sdb/browsing/bmovie_grade.nhn 등급

In [4]:
def get_category(bs,main_url):
    #data = bs.table.findAll('a')
    data = bs.findAll('a',{'href':re.compile('bmovie.nhn*')})
    category = {}
    for item in data:
        name = item.get_text() # 카테고리 내용
        name = name.replace(' ','') # 공백 제거
        #print(name,end='')
        url = main_url+item.attrs['href'] # 카테고리 주소 
        #print(' : ',url)
        category[name]=url 
        
    return category
        

In [51]:
def get_movie(url):
    main_url = 'https://movie.naver.com'
    page = '&page=' # 페이지 설정
    page_num = 1
    movie ={}
    while True:
        bs = getBs(url+page+str(page_num))
        
            
            
        movies = bs.find('ul',{'class':'directory_list'}).findAll('li') # 영화 리스트 담긴 테그 추출
        
        for item in movies:
            m = item.find('a',{'href':re.compile('/movie*')}) # 쓸대 없는 테그 제거
            if m is None: # 원하는 정보가 없는 테그
                pass
            else:
                #print(m)
                movie[m.get_text()] = main_url+m.attrs['href']
        
        if bs.find('td',{'class':'next'}) is None: # 다음 페이지 유무 체크
            break
        page_num+=1
    
    
    return movie
    
    
    

In [47]:
def get_txtfile(rev,filename):
    text = ' '.join(rev)
    path = filename+'/'
    f = open(path+filename+'.txt','w',-1, "utf-8")
    
    f.write(text)
    f.close()

In [56]:
def get_rev(url,limit):
    code = url[url.find('code='):]
    sub_url_front= 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?'
    sub_url_back = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
    page = 1
    target = sub_url_front+code+sub_url_back+str(page)
    bs = getBs(target)
    count = 0
    
    rev = list()
    while True:
        print('craw'+str(page)+'page')
        review_data = bs.findAll('span',{'id':re.compile('_filtered_ment_*')})
    
    
        for i in review_data:
            #print(i.get_text().strip())
            rev.append(i.get_text().strip())
        
        count+=1
        page +=1
        target = sub_url_front+code+sub_url_back+str(page)
        bs = getBs(target)
        
        if bs.find('a',{'title':'다음'}) is None:
            print('no data')
            break
        elif count >= limit:
            print('limit')
            break
    
    
    
    return rev
    

In [25]:
def get_title(code):
    url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+code
    
    bs = getBs(url)
    title = bs.find('h3',{'class':'h_movie'}).a.get_text()
    
    return title

get_title('167613')

'조커'

In [29]:
def get_postimg(code):
    #code = 179482
    url = 'https://movie.naver.com/movie/bi/mi/photoViewPopup.nhn?movieCode='+code
    title = get_title(code)
    bs = getBs(url)
    img_src = bs.find('img',{'id':'targetImage'})
    downlink = img_src.attrs['src']
    urlretrieve(downlink,title+'/'+title+'.jpg')

In [61]:
def get_data(url):
    code = url[url.find('code=')+5:]
    print('code : '+code)
    
    title = get_title(code)
    print('title : '+title)
    
    os.makedirs(title)
    
    get_postimg(code)
    rev = get_rev(url,1000)
    
    get_txtfile(rev,title)
    
    
    

In [ ]:
get_data('https://movie.naver.com/movie/bi/mi/basic.nhn?code=156464')

code : 156464
title : 보헤미안 랩소디
craw1page
craw2page
